In [ ]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [ ]:
start_index = 20
end_index = 23
cam_name = 'cam2'

In [ ]:
from services.label_service import get_map_not_zero_in_sphere

map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, 8, False)

In [ ]:
from services.label_service import build_custom_voxel_pcd

voxel_pcd, voxel_points2instances, voxel_trace = build_custom_voxel_pcd(map, points2instances)

import open3d as o3d
from utils.pcd_utils import color_pcd_by_labels

voxel_pcd_colored = color_pcd_by_labels(voxel_pcd, voxel_points2instances[:, 1])
o3d.visualization.draw_geometries([voxel_pcd_colored])

In [ ]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

points = np.asarray(voxel_pcd.points)
spatial_distance = cdist(points, points)

print("len(voxel_pcd.points) = {}".format(len(voxel_pcd.points)))
print("len(voxel_points2instances) = {}".format(len(voxel_points2instances)))

dist, masks = sam_label_distance(voxel_points2instances, spatial_distance, 2, 10)

In [ ]:
from services.normalized_cut_service import normalized_cut

T = 0.3
clusters = normalized_cut(dist, np.asarray(points), T)

In [ ]:
len(clusters)

In [ ]:
import numpy as np
from scipy.spatial import KDTree

cloud1 = np.asarray(map.points)
cloud2 = np.asarray(voxel_pcd.points)

# Build KD-trees for each point cloud
tree1 = KDTree(cloud1)
tree2 = KDTree(cloud2)

# Find nearest neighbors for each point in cloud1
dists, indices = tree2.query(cloud1, k=1)

# Get the neighbor points from cloud2
neighbors = cloud2[indices]

In [ ]:
from utils.pcd_utils import color_pcd_by_clusters_common

import open3d as o3d

pcd_clustered1, pcd_clustered2, counter1, counter2 = color_pcd_by_clusters_common(map, clusters, neighbors)

print("counter1 = {}, counter2 = {}".format(counter1, counter2))
print("removed points = {}".format(len(map.points) - len(voxel_pcd.points)))
print("len(map.points) = {}".format(len(map.points)))
print("len(voxel_pcd.points) = {}".format(len(voxel_pcd.points)))

o3d.visualization.draw_geometries([pcd_clustered1])
o3d.visualization.draw_geometries([pcd_clustered2])